In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [2]:
N=100
epochs=10
mini_batch_size = 20

In [3]:
train_input, train_target, train_classes, \
test_input, test_target, test_classes = \
    prologue.generate_pair_sets(N)

In [4]:
#train_classes = nn.functional.one_hot(train_classes, num_classes=10)
#test_classes = nn.functional.one_hot(test_classes, num_classes=10)

In [41]:
#code from practical 4b
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.MaxPool2d(2, stride=2))
        
        #predict implicitely the class
        self.fc1 = nn.Sequential(
            nn.Linear(32 * 2 * 2, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 10),
            nn.Softmax(1))
        
        #predict the output
        self.fc2 = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(inplace=True),
            nn.Linear(10,2),
            nn.Softmax(1))
        
    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output = self.fc2(torch.cat((output1, output2), 1))
        return output
    
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25):
    #classification
    criterion = nn.CrossEntropyLoss()
    optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    eta=1e-1
            
    for e in range(nb_epochs):
        print(e)
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                           train_input.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
            
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            print(loss)
            
            optim.zero_grad()
            loss.backward()
            optim.step()
        
#def compute_nb_errors(model, input_, target_, mini_batch_size):
#    nb_error = 0
#    
#    for b in range(0, input_.size(0), mini_batch_size):
#        output = model(input_.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
#                       input_.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
#        _,pred_class_0 = output[0].max(1)
#        _,pred_class_1 = output[1].max(1)
#
#        for k in range(mini_batch_size):
#            if target_[b+k,0] != pred_class_0[k]:
#                nb_error += 1
#            if target_[b+k,1] != pred_class_1[k]:
#                nb_error += 1
#                
#    return nb_error

def compute_nb_errors(model, input_, target_, mini_batch_size):
    nb_error = 0
    
    for b in range(0, input_.size(0), mini_batch_size):
        output = model(input_.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                       input_.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
        _,pred = output.max(1)

        for k in range(mini_batch_size):
            if target_[b+k] != pred[k]:
                nb_error += 1
                
    return nb_error

# weight sharing -> siamese network 

In [42]:
model = Net(20)

In [43]:
train_model(model, train_input, train_target, mini_batch_size)

0
tensor(0.6707, grad_fn=<NllLossBackward>)
tensor(0.7124, grad_fn=<NllLossBackward>)
tensor(0.7101, grad_fn=<NllLossBackward>)
tensor(0.7184, grad_fn=<NllLossBackward>)
tensor(0.7050, grad_fn=<NllLossBackward>)
1
tensor(0.6721, grad_fn=<NllLossBackward>)
tensor(0.7104, grad_fn=<NllLossBackward>)
tensor(0.7058, grad_fn=<NllLossBackward>)
tensor(0.7065, grad_fn=<NllLossBackward>)
tensor(0.7008, grad_fn=<NllLossBackward>)
2
tensor(0.6775, grad_fn=<NllLossBackward>)
tensor(0.7022, grad_fn=<NllLossBackward>)
tensor(0.6993, grad_fn=<NllLossBackward>)
tensor(0.6967, grad_fn=<NllLossBackward>)
tensor(0.6944, grad_fn=<NllLossBackward>)
3
tensor(0.6807, grad_fn=<NllLossBackward>)
tensor(0.6949, grad_fn=<NllLossBackward>)
tensor(0.6936, grad_fn=<NllLossBackward>)
tensor(0.6895, grad_fn=<NllLossBackward>)
tensor(0.6880, grad_fn=<NllLossBackward>)
4
tensor(0.6834, grad_fn=<NllLossBackward>)
tensor(0.6900, grad_fn=<NllLossBackward>)
tensor(0.6840, grad_fn=<NllLossBackward>)
tensor(0.6813, grad_fn=<

In [44]:
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                  nb_test_errors, test_input.size(0)))

test error Net 43.00% 43/100
